In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
LABELS = ['THEORETICAL', 'ENGINEERING', 'EMPIRICAL', 'OTHERS']

In [3]:
test_data = pd.read_csv('./data/task2_private_testset.csv')

In [4]:
ensem_idx = range(5)

In [5]:
scores = []
thres = []
preds = []
test_results = []
for idx in ensem_idx:
    model_dir = 'model_' + str(idx) + '/'
    thres.append(np.loadtxt(os.path.join(model_dir, 'thres.txt')))
    test_result = np.loadtxt(os.path.join(model_dir, 'test_results.csv'), delimiter=',')
    test_results.append(test_result)
    preds.append((test_result >= thres).astype(np.float32))

In [6]:
avg_score = np.mean(test_results, axis=0)
avg_pred = np.mean(preds, axis=0)
pred = (avg_score >= 0.5).astype(np.int32)

In [7]:
for i in range(test_data.shape[0]):
    if np.sum(pred[i]) == 0:
        pred[i][np.argmax(avg_score[i])] = 1

In [8]:
def SubmitGenerator(prediction, sampleFile, public=True, filename='prediction.csv'):
    """
    Args:
        prediction (numpy array)
        sampleFile (str)
        public (boolean)
        filename (str)
    """
    sample = pd.read_csv(sampleFile)
    submit = {}
    submit['order_id'] = list(sample.order_id.values)
    redundant = len(sample) - prediction.shape[0]
    if public:
        submit['THEORETICAL'] = list(prediction[:,0]) + [0]*redundant
        submit['ENGINEERING'] = list(prediction[:,1]) + [0]*redundant
        submit['EMPIRICAL'] = list(prediction[:,2]) + [0]*redundant
        submit['OTHERS'] = list(prediction[:,3]) + [0]*redundant
    else:
        submit['THEORETICAL'] = [0]*redundant + list(prediction[:,0])
        submit['ENGINEERING'] = [0]*redundant + list(prediction[:,1])
        submit['EMPIRICAL'] = [0]*redundant + list(prediction[:,2])
        submit['OTHERS'] = [0]*redundant + list(prediction[:,3])
    df = pd.DataFrame.from_dict(submit) 
    df.to_csv(filename,index=False)

In [9]:
SubmitGenerator(pred, './data/task2_sample_submission.csv', False, 'prediction.csv')